In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [1]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from sklearn.model_selection import KFold
from modules.data_structures.model_dataset import ModelDataset
from model_binaries.utils.binary_utils import save_entity, load_entity, cross_val_train, compute_f1, graph_entity, evaluate_modelV01_predictions

2025-01-30 09:11:03.725629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_000.yaml'
yaml_path1 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_001.yaml'
yaml_path2 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_002.yaml'
yaml_path3 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_003.yaml'


yaml_paths = [yaml_path0, yaml_path1, yaml_path3]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train Datasets into Memory

In [3]:
# File path to load train, test, and val datasets
train_test_val_folder_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v01"

In [4]:
train_dataset = load_entity(train_test_val_folder_path, "train.pkl")
validation_dataset = load_entity(train_test_val_folder_path, "val.pkl")

Load & Train on predefined trained and val datasets

In [5]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path3]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [6]:
models = model_manager.create_models(yamls)

# If no val dataset, code requires to push None instead
val_dataset = validation_dataset if validation_dataset is not None else None

model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)

INFO:root:Training model '43834563003bc611e0b4eb44ff245a3c' for 20 epochs with batch size 32.


Epoch 1/20
637/637 [==============================] - 16s 22ms/step - loss: 353.7422 - mse: 353.7422 - mae: 13.3475 - val_loss: 280.8993 - val_mse: 280.8993 - val_mae: 13.5919
Epoch 2/20
637/637 [==============================] - 15s 24ms/step - loss: 180.7775 - mse: 180.7775 - mae: 10.6604 - val_loss: 158.1165 - val_mse: 158.1165 - val_mae: 10.0338
Epoch 3/20
637/637 [==============================] - 13s 21ms/step - loss: 175.1431 - mse: 175.1431 - mae: 10.4912 - val_loss: 174.6031 - val_mse: 174.6031 - val_mae: 10.5174
Epoch 4/20
637/637 [==============================] - 15s 24ms/step - loss: 169.7074 - mse: 169.7074 - mae: 10.3553 - val_loss: 174.9150 - val_mse: 174.9150 - val_mae: 10.5896
Epoch 5/20
637/637 [==============================] - 16s 26ms/step - loss: 168.1547 - mse: 168.1547 - mae: 10.3031 - val_loss: 304.9839 - val_mse: 304.9839 - val_mae: 14.1282
Epoch 6/20
637/637 [==============================] - 15s 24ms/step - loss: 165.0939 - mse: 165.0939 - mae: 10.2251 - va

Assess Performance

In [29]:
predictions = models[0].predict(validation_dataset.examples, return_target_labels=True)

In [ ]:
# Get all metrics for trained model on val dataset predictions
metrics = evaluate_modelV01_predictions(predictions)
metrics

In [ ]:
# Graph distribution of predictions against labels 
graph_entity(predictions=predictions,bins=50)

In [ ]:
# Print actual predictions against labels
predictions

Launch Tensorboard

In [ ]:
%tensorboard --logdir=./logs

CROSS VALIDATION ALTERNATIVE

Create and Train Models Using KFold Cross Validation

In [24]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path3]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

Get Averaged Metrics for KFold

In [ ]:
avg_metrics = cross_val_train(model_manager=model_manager, yamls=yamls, train_dataset=train_dataset, n_splits=5)
f1 = compute_f1(avg_metrics["val_precision"], avg_metrics["val_recall"])
print(avg_metrics)
print(f1)